In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np 

# SCRAPPING

In [5]:
base_url = 'https://www.amazon.in/'
current = 'https://www.amazon.in/ASUS-15-6-inch-Integrated-Transparent-X515MA-BR011W/product-reviews/B09SGGB687/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

title=[]
rating=[]
name=[]
review=[]
response = requests.get(current)
i=2
while response.status_code==200 and i<37:
  page = bs(response.text,'html.parser')
  data = page.find_all('div',{'data-hook':'review'})

  for item in data:
    title.append(item.find('a',{'data-hook':'review-title'}).text.strip())
    stars=item.find(class_='a-icon-alt').text
    stars = int(float(re.search('[\d.]+',stars).group()))
    rating.append(stars)
    name.append(item.find(class_='a-profile-name').text)
    review.append(item.find(class_='review-text-content').text)

  next_page_url = ('https://www.amazon.in//ASUS-15-6-inch-Integrated-Transparent-X515MA-BR011W/product-reviews/B09SGGB687?pageNumber='+str(i)+'&reviewerType=all_reviews')
  current=next_page_url
  i=i+1
  response = requests.get(current)
  while response.status_code != 200:
        response = requests.get(current)

In [6]:
df = pd.DataFrame()

In [7]:
df['User'] = name
df['Title Review']=title
df['Rating']=rating
df['Review']=review

In [10]:
df.to_csv('ASUS VivoBook 15.csv',index=False)

In [108]:
df = pd.read_csv('/content/ASUS VivoBook 15.csv')

#CLEANING

In [127]:
df=df.dropna()

In [128]:
customer_reviews=[]
i = 0
index=[]
for txt in df['Review']:
  line = txt.replace('\n','')
  line = line.replace('The media could not be loaded.','')
  line = re.sub(' +', ' ', line)
  line = str(line)
  line = line.lstrip(' ')
  if len(line) != 0:
    customer_reviews.append((line))
    index.append(i)
  i+=1

In [ ]:
clean_df = df.iloc[index,:]
clean_df

In [ ]:
clean_df['Review']=customer_reviews

In [137]:
clean_df

,User,Title Review,Rating,Review
0,Moumita Paul,Very good laptoo at 24k price,5,This is such a premium looking laptop at this ...
1,Kesav,Budget laptop,4,Thin and light but delicate too. Bought it for...
2,Anoopsingh,Overall a good deal,5,I am using this laptop for a week and feeling ...
3,Madhu R.,Lies of Amazon Prime Day sale. Fooling custome...,1,"The MRP shown was around Rs. 45,000/-, as per ..."
4,Golden Bieber,Highly recommended laptop for those whose bud...,5,I received this laptop 💻 two days ago.I like ...
...,...,...,...,...
344,Sandip Khanal,It's good under 25k,4,I love this laptop in this budget 25K🔥💰
345,Chirag,Front camera cleartiy is so poor,4,Good product but camera quality is so poor
346,Swapnil vyas,GOOD,5,Good laptop for this price range
347,Vinod Chouhan,Best in this prize range,5,Nothing


In [138]:
clean_df.to_csv('clean_scrap_vivo15.csv',index=False)